In [10]:
from pynq import Overlay
import pynq.lib.dma
# Load the overlay
overlay = Overlay('/home/xilinx/pynq/overlays/bicubic/bicubic.bit')
#overlay?
# Load the DMA
dma = overlay.dma  
# Load bicubic IP
bicubic_ip = overlay.bicubic
#bicubic_ip?

In [11]:
import cv2
import numpy as np
in_img = cv2.imread('input_image.jpeg')
height,width, nr_of_channels = in_img.shape

In [12]:
def padding(img,H,W,C):
    zimg = np.zeros((H+4,W+4,C))
    zimg[2:H+2,2:W+2,:C] = img
    #Pad the first/last two col and row
    zimg[2:H+2,0:2,:C]=img[:,0:1,:C]          # the leftmost two col
    zimg[H+2:H+4,2:W+2,:]=img[H-1:H,:,:]      # the top two rows
    zimg[2:H+2,W+2:W+4,:]=img[:,W-1:W,:]      # the rightmost two col     
    zimg[0:2,2:W+2,:C]=img[0:1,:,:C]          # the bottom two rows          
    #Pad the missing eight points                        
    zimg[0:2,0:2,:C]=img[0,0,:C]              # down left corner (4 points)               
    zimg[H+2:H+4,0:2,:C]=img[H-1,0,:C]        # up left corner   (4 points)                    
    zimg[H+2:H+4,W+2:W+4,:C]=img[H-1,W-1,:C]  # up right corner  (4 points)                
    zimg[0:2,W+2:W+4,:C]=img[0,W-1,:C]        # down right corner(4 points)  
    return zimg

In [13]:
do_padding = 0

In [14]:
if do_padding:
    in_img = padding(in_img,height,width,nr_of_channels)
    cv2.imwrite('padding_image.jpeg', in_img)
    height,width, nr_of_channels = in_img.shape

In [15]:
# Assign input constant values
interpolation_parameter = 5
scale_factor = 2
bicubic_ip.write(0x10,height)                            
bicubic_ip.write(0x18,width)                            
bicubic_ip.write(0x20,interpolation_parameter)                       
bicubic_ip.write(0x28,scale_factor) 

In [16]:
from   pynq import allocate
import time
size_stream_in  = height * width
out_height = scale_factor * height
out_width = scale_factor * width
if do_padding:
    size_stream_out = scale_factor * (height - 4) * scale_factor * (width - 4)
else:
    size_stream_out = out_height * out_width
in_buffer  = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
out_buffer = allocate(shape=(size_stream_out,1), dtype=np.uint32)
k= 0
for i in range(height):
    for j in range(width):
            in_buffer[k,0]=in_img[i,j,0]+256*in_img[i,j,1]+65536*in_img[i,j,2]
            k = k + 1

In [17]:
CPU_C_exec_time = 7.254
CPU_Python_exec_time = 39.52
# Trigger the DMA transfer and wait for the result
start_time = time.time()
dma.sendchannel.transfer(in_buffer)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(out_buffer) #accelerator PYNQ - dma - python
dma.sendchannel.wait()               #se asigura ca nu se trece mai departe pana nu se face transferul respectiv
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time - start_time
print('Hardware bicubic interpolation execution time: ',hw_exec_time)
print('Hardware acceleration factor (FPGA accelerator vs C/C++ CPU version): ',CPU_C_exec_time / hw_exec_time)
print('Hardware acceleration factor (FPGA accelerator vs Python CPU version): ',CPU_Python_exec_time / hw_exec_time)

Hardware bicubic interpolation execution time:  0.3131232261657715
Hardware acceleration factor (FPGA accelerator vs C/C++ CPU version):  23.16659830324959
Hardware acceleration factor (FPGA accelerator vs Python CPU version):  126.21229183132395


In [18]:
out_img = np.zeros((out_height, out_width, nr_of_channels))
k = 0
for i in range(out_height):
    for j in range(out_width):
            binn = np.binary_repr(out_buffer[k,0], width = 32)
            out_img[i,j,0] = int(str(binn[24:32]), 2)
            out_img[i,j,1] = int(str(binn[16:24]), 2)
            out_img[i,j,2] = int(str(binn[8:16]), 2)
            k = k+1
cv2.imwrite('output_image.jpeg', out_img)

True